In [1]:
# from services.openai.wrappers.wrapper_context import OpenAIFunctionContext
from services.openai.wrappers.wrapper_no_context import OpenAIFunctionNoContext
# from services.openai.wrappers.audio_generator import OpenAITextToSpeech

import importlib
import os
import openai
from TTS.api import TTS

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
user_input = "I need to study the Spanish Civil War"
hours_max = 2

In [4]:
params = importlib.import_module("services.openai.prompts.sections_generator")
params.system_template = params.system_template.format(hours=hours_max)


class SectionsGenerator(OpenAIFunctionNoContext):
    system_template = params.system_template
    user_template = params.user_template
    function_name = params.function_name
    function_description = params.function_description
    function_schema = params.function_schema


openai_answer = SectionsGenerator(
    model_name=params.model_name,
    temperature=params.temperature,
    top_p=params.top_p,
    timer=params.timer,
)

In [5]:
answer_sections = openai_answer.predict_on_text(user_input)

answer_sections

[{'topic_title': 'Spanish Civil War',
  'section': [{'section_number': 1,
    'section_title': 'Introduction to the Spanish Civil War',
    'section_words': 2000},
   {'section_number': 2,
    'section_title': 'Causes of the Spanish Civil War',
    'section_words': 3000},
   {'section_number': 3,
    'section_title': 'Major Events of the Spanish Civil War',
    'section_words': 4000},
   {'section_number': 4,
    'section_title': 'Key Figures in the Spanish Civil War',
    'section_words': 3000},
   {'section_number': 5,
    'section_title': 'International Involvement in the Spanish Civil War',
    'section_words': 3000},
   {'section_number': 6,
    'section_title': 'Consequences of the Spanish Civil War',
    'section_words': 2000},
   {'section_number': 7,
    'section_title': 'Legacy of the Spanish Civil War',
    'section_words': 2000}]}]

In [6]:
topic_title = answer_sections[0]["topic_title"]
section_title = answer_sections[0]["section"][0]["section_title"]
section_words = answer_sections[0]["section"][0]["section_words"]

In [7]:
params = importlib.import_module("services.openai.prompts.content_generator")
params.system_template = params.system_template.format(
    topic_field=hours_max, words_amount=section_words
)


class ContentGenerator(OpenAIFunctionNoContext):
    system_template = params.system_template
    user_template = params.user_template
    function_name = params.function_name
    function_description = params.function_description
    function_schema = params.function_schema


openai_answer = ContentGenerator(
    model_name=params.model_name,
    temperature=params.temperature,
    top_p=params.top_p,
    timer=params.timer,
)

In [8]:
answer_content = openai_answer.predict_on_text(section_title)

answer_content

[{'section_content': 'The Spanish Civil War, which lasted from 1936 to 1939, was a significant conflict that not only shaped the future of Spain but also had profound implications for Europe and the world at large. This war was fought between the Republicans, who were a coalition of leftist groups including communists, socialists, and anarchists, and the Nationalists, led by General Francisco Franco, who represented conservative, monarchist, and fascist elements. Understanding the Spanish Civil War requires an exploration of its causes, key events, and consequences, as well as its legacy in both Spanish and global contexts.\n\n### Historical Context\n\nTo comprehend the Spanish Civil War, one must first consider the historical context of Spain in the early 20th century. The country was marked by political instability, social unrest, and economic challenges. The aftermath of World War I had left many European nations in turmoil, and Spain was no exception. The monarchy, which had ruled 

In [9]:
text_to_audio = answer_content[0]["section_content"]
save_path = "output.wav"

In [10]:
# Initialize TTS with the selected model
tts = TTS(
    model_name="tts_models/en/ljspeech/tacotron2-DDC",
    vocoder_path="vocoder_models/en/ljspeech/hifigan_v2",
    progress_bar=False,
    gpu=False,  # Set to False if you don't have a GPU
)

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

In [11]:
# Generate speech and save to a file
tts.tts_to_file(text=text_to_audio, file_path=save_path)

 > Text splitted to sentences.
['The Spanish Civil War, which lasted from 1936 to 1939, was a significant conflict that not only shaped the future of Spain but also had profound implications for Europe and the world at large.', 'This war was fought between the Republicans, who were a coalition of leftist groups including communists, socialists, and anarchists, and the Nationalists, led by General Francisco Franco, who represented conservative, monarchist, and fascist elements.', 'Understanding the Spanish Civil War requires an exploration of its causes, key events, and consequences, as well as its legacy in both Spanish and global contexts.', '### Historical Context', 'To comprehend the Spanish Civil War, one must first consider the historical context of Spain in the early 20th century.', 'The country was marked by political instability, social unrest, and economic challenges.', 'The aftermath of World War I had left many European nations in turmoil, and Spain was no exception.', 'The 

'output.wav'